In [1]:
import pandas as pd 
import numpy as np 


In [2]:
training_data = pd.read_csv('data/liar-plus/train2.tsv', sep='\t', header=None)
training_data.columns =['index','id', 'label', 'statement', 'subject', 'speaker', 'speaker_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context','justification']
training_data.drop(columns=['index'], inplace=True)
training_data.head()

,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."


In [3]:
df = training_data[['statement', 'party_affiliation', 'label']]
df

,statement,party_affiliation,label
0,Says the Annies List political group supports ...,republican,false
1,When did the decline of coal start? It started...,democrat,half-true
2,"Hillary Clinton agrees with John McCain ""by vo...",democrat,mostly-true
3,Health care reform legislation is likely to ma...,none,false
4,The economic turnaround started at the end of ...,democrat,half-true
...,...,...,...
10237,There are a larger number of shark attacks in ...,none,mostly-true
10238,Democrats have now become the party of the [At...,republican,mostly-true
10239,Says an alternative to Social Security that op...,republican,half-true
10240,On lifting the U.S. Cuban embargo and allowing...,democrat,false


In [4]:
df['word_count'] = df['statement'].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)
df

/var/folders/pd/jy2tczp11tvgftzcy6kn7txh0000gn/T/ipykernel_48040/1403504826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_count'] = df['statement'].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)


,statement,party_affiliation,label,word_count
0,Says the Annies List political group supports ...,republican,false,11
1,When did the decline of coal start? It started...,democrat,half-true,24
2,"Hillary Clinton agrees with John McCain ""by vo...",democrat,mostly-true,19
3,Health care reform legislation is likely to ma...,none,false,12
4,The economic turnaround started at the end of ...,democrat,half-true,10
...,...,...,...,...
10237,There are a larger number of shark attacks in ...,none,mostly-true,17
10238,Democrats have now become the party of the [At...,republican,mostly-true,14
10239,Says an alternative to Social Security that op...,republican,half-true,28
10240,On lifting the U.S. Cuban embargo and allowing...,democrat,false,11


In [5]:
df['word_count'].describe()

count    10242.000000
mean        18.027338
std          9.928149
min          0.000000
25%         12.000000
50%         17.000000
75%         22.000000
max        479.000000
Name: word_count, dtype: float64

In [6]:
datum = df.iloc[0]

In [7]:
import spacy
nlp = spacy.load("en_core_web_md")
doc = nlp(datum['statement'])
doc.vector.shape

(300,)

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

def chunk_text(text, chunk_size=200, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

model = SentenceTransformer("all-MiniLM-L6-v2")  # small but high quality

def embed_statement(text):
    chunks = chunk_text(text)
    embeddings = model.encode(chunks)
    return np.mean(embeddings, axis=0)  # average chunk embeddings

df["embedding"] = df["statement"].apply(embed_statement)
X = np.stack(df["embedding"])
y = df["truth_value"]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
